In [ ]:
%cd /content
import IPython.display
import os
import sys
import random
import string
import urllib.parse
import subprocess
import threading
import time
import re
import socket

if not os.path.exists("/content/lineworks"):
    !git clone https://github.com/arenasys/lineworks.git

%cd /content/lineworks
!git pull

if not os.path.exists("venv"):
    %cd /content/lineworks/models

    ### Model URL Here
    !wget https://huggingface.co/TheBloke/Mistral-7B-v0.1-GGUF/resolve/main/mistral-7b-v0.1.Q5_K_M.gguf
    %cd /content/lineworks

    !mkdir venv
    !pip uninstall -q -y tensorflow
    !pip install -q websockets==11.0.3 bson==0.5.10 cryptography==40.0.2 
    !pip install https://github.com/jllllll/llama-cpp-python-cuBLAS-wheels/releases/download/wheels/llama_cpp_python-0.2.11+cu118-cp310-cp310-manylinux_2_31_x86_64.whl
    IPython.display.clear_output()

    !apt -y update -qq
    !wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
    %env LD_PRELOAD=/content/libtcmalloc_minimal.so.4
    IPython.display.clear_output()

    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
    !chmod +x cloudflared
    IPython.display.clear_output()

IPython.display.clear_output()
print("STARTING...")
password = ''.join(random.SystemRandom().choice(string.ascii_letters + string.digits) for _ in range(8))
stopping = False

def tunnel_thread():
    while True:
        time.sleep(0.1)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', 29999))
        if result == 0:
            break
        sock.close()
    time.sleep(1)

    url_pattern = re.compile(r"(?P<url>https?://\S+\.trycloudflare\.com)")
    p = subprocess.Popen(["./cloudflared", "tunnel", "--url", "http://127.0.0.1:29999"], stdout=subprocess.DEVNULL, stderr=subprocess.PIPE, encoding="utf-8")
    for line in p.stderr:
        url_match = url_pattern.search(line)
        if url_match:
            tunnel_url = url_match.group("url")
            endpoint = tunnel_url.replace("https", "wss").strip()
            print("ENDPOINT:", endpoint)
            print("READY!")
            break
    while not stopping:
        time.sleep(0.1)
    print("DIE")
    p.kill()
            
print("PASSWORD:", password)
threading.Thread(target=tunnel_thread, daemon=True).start()
!python source/server.py --password "$password"
stopping = True